<a href="https://colab.research.google.com/github/Gaulgeous/Energy-Forecasting/blob/main/colab_trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import InputLayer, LSTM, Dense, Conv1D, Flatten, GRU, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam

import absl.logging
from sklearn.metrics import mean_squared_error as mse, r2_score, mean_absolute_error as mae, mean_absolute_percentage_error as mape
from sklearn.model_selection import TimeSeriesSplit
from statsmodels.tsa.seasonal import seasonal_decompose

import time
import os
import pandas as pd
import numpy as np
import keras_tuner as kt
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from skopt import BayesSearchCV
from skopt.space import Categorical
from skopt import dump, load

import xgboost as xgb

import math

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [29]:
def build_baseline_model():

  model = Sequential()
  model.add(Dense(16, activation='relu'))
  model.add(Dense(1, 'linear'))

  model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.001), 
              metrics=['mean_squared_error'])

  return model


def build_simple_model():

  model = Sequential()
  model.add(Dense(64, activation='relu'))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(16, activation='relu'))

  model.add(Dense(1, 'linear'))

  model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.001), 
              metrics=['mean_squared_error'])

  return model


def train_simple_model(model, X_frame, y_frame, split, data_epochs, batch_size, y_scaler):

  length = X_frame.shape[0]
  X_train = X_frame[:int(length*split),:]
  y_train = y_frame[:int(length*split)]

  X_test = X_frame[int(length*split):,:]
  y_test = y_frame[int(length*split):]

  model.fit(X_train, y_train, verbose=0, epochs=data_epochs,
                  batch_size=batch_size, validation_split=0.2)
  preds = model.predict(X_test, verbose=0)
  preds = y_scaler.inverse_transform(preds)
  y_test = y_scaler.inverse_transform(y_test)

  return mse(y_test, preds, squared=True)


def simple_evaluate(future, set_name, X_train, y_train, epochs, batch_size):

  time_start = time.time()

  model_directory = ""

  absl.logging.set_verbosity(absl.logging.ERROR)
  tf.compat.v1.logging.set_verbosity(30)

  model = build_baseline_model()
  model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=0)

  model.save(model_directory + "" + set_name + "_baseline_" + str(future))
  
  print("Finished evaluating baseline for future {0}".format(future))

  time_end = time.time()

  return time_end - time_start


def simple_predict(future, set_name, pred_dates_test, X_test, y_test, y_scaler):

  folder_path = os.getcwd()
  model_directory = "" 
  csv_directory = ""

  model = load_model(model_directory + "" + set_name + "_baseline_" + str(future))
  predictions = model.predict(X_test)
  predictions = y_scaler.inverse_transform(predictions).reshape(-1)
  y_test = y_scaler.inverse_transform(y_test).reshape(-1)

  make_csvs(csv_directory, predictions, y_test, pred_dates_test, set_name, future, "Baseline")

  print("Finished running baseline prediction on future window {0}".format(future))

  metric_outputs = get_metrics(predictions, y_test, 0, "Baseline")
  return metric_outputs



In [30]:
def get_metrics(predictions, actual, cv, model_name):

    MSE = mse(actual, predictions, squared=True)
    MAE = mae(actual, predictions)
    MAPE = mape(actual, predictions)
    RMSE = mse(actual, predictions, squared=False)
    R2 = r2_score(actual, predictions)
    if cv:
        metrics = {model_name + '_RMSE': RMSE, model_name + '_R2': R2, model_name +'_MSE': MSE, 
                   model_name + '_MAE': MAE, model_name + '_MAPE': MAPE}
    else:
        metrics = {'RMSE': RMSE, 'R2': R2, 'MSE': MSE, 'MAE': MAE, 'MAPE': MAPE}
    return metrics


def cross_val_metrics(total_metrics, set_name, future, model_name):

    csv_directory = ""
    df = pd.DataFrame(total_metrics)
    df.to_csv(csv_directory + "" + set_name + "_" + model_name + "_cv_metrics_" + str(future) + ".csv", index=False)


def normalise_metrics(metrics, training):

    rmse = [key["RMSE"] for key in metrics]
    mse = [key["MSE"] for key in metrics]
    mae = [key["MAE"] for key in metrics]
    r2 = [key["R2"] for key in metrics]

    metrics_sets = {"RMSE": rmse, "MSE": mse, "MAE": mae, "R2": r2}

    if training:
        time = [key["TIME"] for key in metrics]
        metrics_sets = {"RMSE": rmse, "MSE": mse, "MAE": mae, "R2": r2, "TIME": time}

    for name, set in metrics_sets.items():
        top = max(set)
        counter = 0

        while top > 10:
            top /= 10
            set = [entry / 10 for entry in set]
            counter += 1
        
        i = 0

        for key in metrics:
            key[name] = set[i]
            i += 1

    return metrics

    
# This is going to have to be rejigged
def make_metrics_csvs(csv_directory, metrics, set_name, future, training):

    for model_name, metric_outputs in metrics.items():
 
        if not os.path.exists(csv_directory + "" + set_name + "_metrics_" + str(future) + ".csv"):
            metrics = pd.DataFrame({"Model": [], "Metric": [], "Value": []})
            metrics.loc[len(metrics)] = {"Model": model_name, "Metric": "RMSE", "Value": metric_outputs.get("RMSE")}
            metrics.loc[len(metrics)] = {"Model": model_name, "Metric": "MAE", "Value": metric_outputs.get("MAE")}
            metrics.loc[len(metrics)] = {"Model": model_name, "Metric": "MAPE", "Value": metric_outputs.get("MAPE")}
            metrics.loc[len(metrics)] = {"Model": model_name, "Metric": "R2", "Value": metric_outputs.get("R2")}
            if training:
                metrics.loc[len(metrics)] = {"Model": model_name, "Metric": "TIME", "Value": metric_outputs.get("TIME")}

            metrics.to_csv(csv_directory + "" + set_name + "_metrics_" + str(future) + ".csv", index=False)
        else:

            metrics = pd.read_csv(csv_directory + "" + set_name + "_metrics_" + str(future) + ".csv")

            if model_name in metrics['Model'].values:
                metrics.loc[(metrics['Model'] == model_name) & (metrics["Metric"] == "RMSE"), 'Value'] = metric_outputs.get("RMSE")
                metrics.loc[(metrics['Model'] == model_name) & (metrics["Metric"] == "MAE"), 'Value'] = metric_outputs.get("MAE")
                metrics.loc[(metrics['Model'] == model_name) & (metrics["Metric"] == "MAPE"), 'Value'] = metric_outputs.get("MAPE")
                metrics.loc[(metrics['Model'] == model_name) & (metrics["Metric"] == "R2"), 'Value'] = metric_outputs.get("R2")
                if training:
                    metrics.loc[(metrics['Model'] == model_name) & (metrics["Metric"] == "TIME"), 'Value'] = metric_outputs.get("TIME")
            else:
                metrics.loc[len(metrics)] = {"Model": model_name, "Metric": "RMSE", "Value": metric_outputs.get("RMSE")}
                metrics.loc[len(metrics)] = {"Model": model_name, "Metric": "MAE", "Value": metric_outputs.get("MAE")}
                metrics.loc[len(metrics)] = {"Model": model_name, "Metric": "MAPE", "Value": metric_outputs.get("MAPE")}
                metrics.loc[len(metrics)] = {"Model": model_name, "Metric": "R2", "Value": metric_outputs.get("R2")}
                if training:
                    metrics.loc[len(metrics)] = {"Model": model_name, "Metric": "TIME", "Value": metric_outputs.get("TIME")}
            metrics.to_csv(csv_directory + "" + set_name + "_metrics_" + str(future) + ".csv", index=False)


def make_csvs(csv_directory, predictions, y_test, pred_dates_test, set_name, future, model_name):

    if not os.path.exists(csv_directory + "" + set_name + "_performances_" + str(future) + ".csv"):
        performances = pd.DataFrame({"Date":pred_dates_test, "Actual": y_test, model_name: predictions})
        performances = performances.iloc[-1000:,:]
        performances.to_csv(csv_directory + "" + set_name + "_performances_" + str(future) + ".csv", index=False)
    else:
        performances = pd.read_csv(csv_directory + "" + set_name + "_performances_" + str(future) + ".csv")
        performances[model_name] = predictions[-1000:]
        performances.to_csv(csv_directory + "" + set_name + "_performances_" + str(future) + ".csv", index=False)

   

In [31]:
def collapse_columns(data):
    data = data.copy()
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = data.columns.to_series().apply(lambda x: "__".join(x))
    return data
    
def create_dataset_2d(input, win_size):
    
    np_data = np.array(input.copy())

    X = []

    for i in range(len(np_data)-win_size):
        row = [r for r in np_data[i:i+win_size]]
        X.append(row)

    X = np.array(X)
    X = X.reshape(X.shape[0], -1)

    return X
    

def create_dataset_3d(input, win_size):
    
    np_data = np.array(input.copy())

    X = []

    for i in range(len(np_data)-win_size):
        row = [r for r in np_data[i:i+win_size]]
        X.append(row)

    return np.array(X)


def load_datasets(csv_directory, set_name, future):

    data_name = csv_directory + "" + set_name + "_data_" + str(future) + ".csv"
    output_name = csv_directory + "" + set_name + "_outputs_" + str(future) + ".csv"

    data = pd.read_csv(data_name).set_index("Date")
    outputs = pd.read_csv(output_name).set_index("Date")

    return data, outputs


def finalise_data(data, outputs, target, best_results):

    pred_dates = outputs.index

    pca_dim = best_results.get("pca_dimensions")
    y_scaler = None
    
    if best_results.get("scaler") == "minmax":
        X_scaler = MinMaxScaler(feature_range=(0,1))
        y_scaler = MinMaxScaler(feature_range=(0,1))
        data = X_scaler.fit_transform(data)
        outputs = y_scaler.fit_transform(outputs[[target]])

    elif best_results.get("scaler") == "standard":
        X_scaler = StandardScaler()
        y_scaler = StandardScaler()
        data = X_scaler.fit_transform(data)
        outputs = y_scaler.fit_transform(outputs[[target]])

    if pca_dim == "None":
        pca = PCA()
        data = pca.fit_transform(data)
    elif pca_dim == "mle":
        pca = PCA(n_components="mle")
        data = pca.fit_transform(data)
    elif pca_dim != "NO_PCA":
        pca = PCA(n_components=pca_dim)
        data = pca.fit_transform(data)

    X_frame = np.array(data)
    y_data = np.array(outputs)

    return X_frame, y_data, pred_dates, y_scaler


def data_cleaning_pipeline(data_in, outputs_in, cleaning_parameters, target, split, data_epochs, batch_size, csv_directory):

    best_results = {"MSE": [math.inf], "scaler": [None], "pca_dimensions": [None]}

    for scale_type in cleaning_parameters.get('scalers'):
            for pca_dim in cleaning_parameters.get('pca_dimensions'):

                data = data_in.copy()
                outputs = outputs_in.copy()

                if scale_type == 'minmax':
                    X_scaler = MinMaxScaler(feature_range=(0,1))
                    y_scaler = MinMaxScaler(feature_range=(0,1))
                    data = X_scaler.fit_transform(data)
                    outputs = y_scaler.fit_transform(outputs[[target]])

                elif scale_type == 'standard':
                    X_scaler = StandardScaler()
                    y_scaler = StandardScaler()
                    data = X_scaler.fit_transform(data)
                    outputs = y_scaler.fit_transform(outputs[[target]])

                if pca_dim == None:
                    pca = PCA()
                    data = pca.fit_transform(data)
                elif pca_dim == -math.inf:
                    pca = PCA(n_components="mle")
                    data = pca.fit_transform(data)
                elif pca_dim != math.inf:
                    pca = PCA(n_components=pca_dim)
                    data = pca.fit_transform(data)

                X_frame = np.array(data)
                y_frame = np.array(outputs)
                
                model = build_simple_model()
                mse = train_simple_model(model, X_frame, y_frame, split, data_epochs, batch_size, y_scaler)
                print("Trained scale:{0} dim:{1}".format(scale_type, pca_dim))
                if mse < best_results.get("MSE"):
                    if pca_dim == None:
                        pca_dim = "None"
                    elif pca_dim == math.inf:
                        pca_dim = "NO_PCA"
                    elif pca_dim == -math.inf:
                        pca_dim = "mle"
                    best_results["MSE"][0] = mse
                    best_results["pca_dimensions"][0] = pca_dim
                    best_results["scaler"][0] = scale_type

    results_data = pd.DataFrame.from_dict(best_results)
    results_data.to_csv(csv_directory + "best_data_parameters.csv", index=False)

    best_results = {"MSE": best_results.get("MSE"), "scaler": best_results.get("scaler")[0], "pca_dimensions": best_results.get("pca_dimensions")[0]}
    return best_results




def feature_adder(csv_directory, file_path, target, trend_type, future, epd,  set_name):

    data = pd.read_excel(file_path).set_index("Date")
    data = collapse_columns(data)

    data['PrevDaySameHour'] = data[target].copy().shift(epd)
    data['PrevWeekSameHour'] = data[target].copy().shift(epd*7)
    data['Prev24HourAveLoad'] = data[target].copy().rolling(window=epd*7, min_periods=1).mean()
    data['Weekday'] = data.index.dayofweek

    if 'Holiday' in data.columns.values:
        data.loc[(data['Weekday'] < 5) & (data['Holiday'] == 0), 'IsWorkingDay'] = 1
        data.loc[(data['Weekday'] > 4) | (data['Holiday'] == 1), 'IsWorkingDay'] = 0
    else:
        data.loc[data['Weekday'] < 5, 'IsWorkingDay'] = 1
        data.loc[data['Weekday'] > 4, 'IsWorkingDay'] = 0

    dec_daily = seasonal_decompose(data[target], model=trend_type, period=epd)
    data['IntraDayTrend'] = dec_daily.trend
    data['IntraDaySeasonal'] = dec_daily.seasonal
    data['IntraDayTrend'] = data['IntraDayTrend'].shift(epd)
    data['IntraDaySeasonal'] = data['IntraDaySeasonal'].shift(epd)

    dec_weekly = seasonal_decompose(data[target], model=trend_type, period=epd*7)
    data['IntraWeekTrend'] = dec_weekly.trend
    data['IntraWeekSeasonal'] = dec_weekly.seasonal
    data['IntraWeekTrend'] = data['IntraWeekTrend'].shift(epd*7)
    data['IntraWeekSeasonal'] = data['IntraWeekSeasonal'].shift(epd*7)

    data[target] = y = data[target].shift(-epd*future)
    data = data.dropna(how='any', axis='rows')
    y = data[target].reset_index(drop=True)
    # y = data[target].shift(-epd*future).reset_index(drop=True)
    # y = y.dropna(how='any', axis='rows')

    future_dates = pd.Series(data.index[future*epd:])
    outputs = pd.DataFrame({"Date": future_dates, "{0}".format(target): y})

    # future > 10 needs addressing - it is not yet implemented
    if future > 10:
        data = data[['DryBulb', 'DewPnt', 'Prev5DayHighAve', 'Prev5DayLowAve', 'Hour', 'Weekday', 'IsWorkingDay']]
    else:
        data = data.drop("{0}".format(target), axis=1)

    data_name = csv_directory + "" + set_name + "_data_" + str(future) + ".csv"
    output_name = csv_directory + "" + set_name + "_outputs_" + str(future) + ".csv"

    data.to_csv(data_name)
    outputs.to_csv(output_name, index=False)

    print("Saved future window {0} to csvs".format(future))

    return data, outputs
    




In [32]:
def bnn_kt_model(hp):

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    hp_reg = hp.Float("reg", min_value=1e-4, max_value=1e-2, sampling="log")
    hp_dropout = hp.Float("dropout", min_value=1e-3, max_value=0.5, sampling="linear")
    hp_neuron_pct = hp.Float('NeuronPct', min_value=1e-3, max_value=1.0, sampling='linear')
    hp_neuron_shrink = hp.Float('NeuronShrink', min_value=1e-3, max_value=1.0, sampling='linear')
    
    hp_max_neurons = hp.Int('neurons', min_value=10, max_value=200, step=10)

    neuron_count = int(hp_neuron_pct * hp_max_neurons)
    layers = 0

    model = Sequential()

    while neuron_count > 5 and layers < 5:

        model.add(Dense(units=neuron_count, activation=hp_activation))
        model.add(Dropout(hp_dropout))
        layers += 1
        neuron_count = int(neuron_count * hp_neuron_shrink)

    model.add(Dense(1, 'linear'))

    model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=hp_learning_rate), 
                metrics=['mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error'])

    return model


def bnn_save_plots(history, graphs_directory, set_name, future):

    graph_names = {"Loss": "loss", "MAE": "mean_absolute_error", 
                   "MSE": "mean_squared_error", "MAPE": "mean_absolute_percentage_error"}
    
    for name, value in graph_names.items():
        graph_loc = graphs_directory + "" + set_name + "_basic_nn_" + str(future) + "_" + name + ".png"
        if os.path.exists(graph_loc):
            os.remove(graph_loc)

        val_name = "val_" + value
        plt.plot(history.history[value])
        plt.plot(history.history[val_name])
        plt.title('Basic NN {0} for {1} {2}'.format(name, set_name, future))
        plt.ylabel(name)
        plt.xlabel('epoch')
        plt.legend(['train', 'test'], loc='upper left')
        plt.savefig(graphs_directory + "" + set_name + "_basic_nn_" + str(future) + "_" + name + ".png")
    

def bnn_train_model(future, batch_size, epochs,
                model_directory, set_name, X_train, y_train, y_scaler, epd):

    tuner = kt.Hyperband(bnn_kt_model, objective='mean_absolute_percentage_error', max_epochs=epochs, factor=3, 
                        directory=model_directory + "" + set_name + "_kt_dir", project_name='kt_model_' + str(future), 
                        overwrite=True)

    monitor = EarlyStopping(monitor='mean_absolute_percentage_error', min_delta=1, patience=5, verbose=0, mode='auto', 
                    restore_best_weights=True)

    tuner.search(X_train, y_train, verbose=0, epochs=epochs, validation_split=0.2, batch_size=batch_size,
                callbacks=[monitor])

    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    model = tuner.hypermodel.build(best_hps)

    # Split on a 3 monthly basis
    tss = TimeSeriesSplit(n_splits=10, test_size=epd*90, gap=0)
    fold = 0
    total_metrics = {}

    for train_idx, val_idx in tss.split(X_train, y_train):

        fold_name = "Fold_" + str(fold)
        X_t = X_train[train_idx]
        X_v = X_train[val_idx]
        y_t = y_train[train_idx]
        y_v = y_train[val_idx]
        
        if fold == 9:
            history = model.fit(X_t, y_t, verbose=0, epochs=epochs, callbacks=[monitor],
                    batch_size=batch_size, validation_data=(X_v, y_v))
            graphs_directory = ""
            bnn_save_plots(history, graphs_directory, set_name, future)
            model.save(model_directory + "" + set_name + "_basic_nn_" + str(future))
        
        model.fit(X_t, y_t, verbose=0, epochs=epochs, callbacks=[monitor],
                    batch_size=batch_size)
        preds = model.predict(X_v, verbose=0)
        preds = y_scaler.inverse_transform(preds)
        metrics = get_metrics(preds, y_v, 1, "Basic_nn")
        total_metrics[fold_name] = metrics

        fold += 1

    cross_val_metrics(total_metrics, set_name, future, "Basic_nn")


def bnn_predict(future, set_name, pred_dates_test, X_test, y_test, y_scaler):

    folder_path = os.getcwd()
    model_directory = folder_path + r"\models"
    csv_directory = folder_path + r"\csvs"

    model = load_model(model_directory + "" + set_name + "_basic_nn_" + str(future))
    predictions = model.predict(X_test)
    predictions = y_scaler.inverse_transform(predictions).reshape(-1)
    y_test = y_scaler.inverse_transform(y_test).reshape(-1)

    make_csvs(csv_directory, predictions, y_test, pred_dates_test, set_name, future, "Basic_nn")

    print("Finished running basic prediction on future window {0}".format(future))

    metric_outputs = get_metrics(predictions, y_test, 0, "Basic_nn")
    return metric_outputs


def bnn_evaluate(future, set_name, X_train, y_train, epochs, batch_size, y_scaler, epd):

    time_start = time.time()

    folder_path = os.getcwd()
    model_directory = folder_path + r"\models"

    absl.logging.set_verbosity(absl.logging.ERROR)
    tf.compat.v1.logging.set_verbosity(30)

    bnn_train_model(future, batch_size, epochs,
            model_directory, set_name, X_train, y_train, y_scaler, epd)
    
    print("Finished evaluating basic nn for future {0}".format(future))

    time_end = time.time()

    return time_end - time_start
    


In [33]:
def cnn_kt_model(hp):

    X = np.load("X_train_3d.npy")

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    hp_reg = hp.Float("reg", min_value=1e-4, max_value=1e-2, sampling="log")
    hp_dropout = hp.Float("dropout", min_value=1e-3, max_value=0.5, sampling="linear")
    hp_neuron_pct = hp.Float('NeuronPct', min_value=1e-3, max_value=1.0, sampling='linear')
    hp_neuron_shrink = hp.Float('NeuronShrink', min_value=1e-3, max_value=1.0, sampling='linear')
    
    hp_l_layer_1 = hp.Int('l_layer_1', min_value=1, max_value=100, step=10)
    hp_max_neurons = hp.Int('neurons', min_value=10, max_value=5000, step=10)

    neuron_count = int(hp_neuron_pct * hp_max_neurons)
    layers = 0

    model = Sequential()
    model.add(InputLayer((X.shape[1], X.shape[2])))
    model.add(LSTM(hp_l_layer_1, return_sequences=True, activity_regularizer=regularizers.l1(hp_reg)))
    model.add(Dropout(hp_dropout))
    model.add(Flatten())

    while neuron_count > 20 and layers < 20:

        model.add(Dense(units=neuron_count, activation=hp_activation))
        model.add(Dropout(hp_dropout))
        layers += 1
        neuron_count = int(neuron_count * hp_neuron_shrink)

    model.add(Dense(1, 'linear'))

    model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=hp_learning_rate), 
                metrics=['mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error'])

    return model


def cnn_save_plots(history, graphs_directory, set_name, future):

    graph_names = {"Loss": "loss", "MAE": "mean_absolute_error", 
                   "MSE": "mean_squared_error", "MAPE": "mean_absolute_percentage_error"}
    
    for name, value in graph_names.items():
        graph_loc = graphs_directory + "" + set_name + "_Complex_nn_" + str(future) + "_" + name + ".png"
        if os.path.exists(graph_loc):
            os.remove(graph_loc)

        val_name = "val_" + value
        plt.plot(history.history[value])
        plt.plot(history.history[val_name])
        plt.title('Complex NN {0} for {1} {2}'.format(name, set_name, future))
        plt.ylabel(name)
        plt.xlabel('epoch')
        plt.legend(['train', 'test'], loc='upper left')
        plt.savefig(graphs_directory + "" + set_name + "_Complex_nn_" + str(future) + "_" + name + ".png")
    

def cnn_train_model(future, batch_size, epochs,
                model_directory, set_name, X_train, y_train, y_scaler, epd):

    tuner = kt.Hyperband(cnn_kt_model, objective='mean_absolute_percentage_error', max_epochs=epochs, factor=3, 
                        directory=model_directory + "" + set_name + "_kt_dir", project_name='kt_model_' + str(future), 
                        overwrite=True)

    monitor = EarlyStopping(monitor='mean_absolute_percentage_error', min_delta=1, patience=5, verbose=0, mode='auto', 
                    restore_best_weights=True)

    tuner.search(X_train, y_train, verbose=0, epochs=epochs, validation_split=0.2, batch_size=batch_size,
                callbacks=[monitor])

    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    model = tuner.hypermodel.build(best_hps)

    # Split on a 3 monthly basis
    tss = TimeSeriesSplit(n_splits=10, test_size=epd*90, gap=0)
    fold = 0
    total_metrics = {}

    for train_idx, val_idx in tss.split(X_train, y_train):

        fold_name = "Fold_" + str(fold)
        X_t = X_train[train_idx]
        X_v = X_train[val_idx]
        y_t = y_train[train_idx]
        y_v = y_train[val_idx]
        
        if fold == 9:
            history = model.fit(X_t, y_t, verbose=0, epochs=epochs, callbacks=[monitor],
                    batch_size=batch_size, validation_data=(X_v, y_v))
            graphs_directory = ""
            cnn_save_plots(history, graphs_directory, set_name, future)
            model.save(model_directory + "" + set_name + "_Complex_nn_" + str(future))
        
        model.fit(X_t, y_t, verbose=0, epochs=epochs, callbacks=[monitor],
                    batch_size=batch_size)
        preds = model.predict(X_v, verbose=0)
        preds = y_scaler.inverse_transform(preds)
        metrics = get_metrics(preds, y_v, 1, "Complex_nn")
        total_metrics[fold_name] = metrics

        fold += 1

    cross_val_metrics(total_metrics, set_name, future, "Complex_nn")



def cnn_predict(future, set_name, pred_dates_test, X_test, y_test, y_scaler):

    folder_path = os.getcwd()
    model_directory = folder_path + r"\models"
    csv_directory = folder_path + r"\csvs"

    model = load_model(model_directory + "" + set_name + "_Complex_nn_" + str(future))
    predictions = model.predict(X_test)
    predictions = y_scaler.inverse_transform(predictions).reshape(-1)
    y_test = y_scaler.inverse_transform(y_test).reshape(-1)

    make_csvs(csv_directory, predictions, y_test, pred_dates_test, set_name, future, "Complex_nn")

    print("Finished running complex prediction on future window {0}".format(future))

    metric_outputs = get_metrics(predictions, y_test, 0, "Complex_nn")
    return metric_outputs


def cnn_evaluate(future, set_name, X_train, y_train, epochs, batch_size, y_scaler, epd):

    time_start = time.time()

    folder_path = os.getcwd()
    model_directory = folder_path + r"\models"

    absl.logging.set_verbosity(absl.logging.ERROR)
    tf.compat.v1.logging.set_verbosity(30)

    cnn_train_model(future, batch_size, epochs,
            model_directory, set_name, X_train, y_train, y_scaler, epd)
    
    print("Finished evaluating complex nn for future {0}".format(future))

    time_end = time.time()

    return time_end - time_start

In [34]:
def rf_train_model(future, epochs, model_directory, set_name, X_train, y_train, epd):

    tss = TimeSeriesSplit(n_splits=5, test_size=epd*90, gap=0)
    estimator = RandomForestRegressor()

    search_space = {
        "max_depth": (10, 1200),
        "min_samples_leaf": (0.001, 0.5, "uniform"),
        "min_samples_split": (0.001, 1.0, "uniform"),
        "n_estimators": (5, 5000),
        "criterion": Categorical(["squared_error"]),
        "max_features": Categorical(['sqrt', 'log2', None]),
    }

    model = BayesSearchCV(
        estimator=estimator,
        search_spaces=search_space,
        scoring="neg_root_mean_squared_error",
        cv=tss,
        n_jobs=-1,
        n_iter=epochs,
        verbose=0,
        refit=True,
    )

    model = model.fit(X_train, y_train)
    dump(model, model_directory + "" + set_name + "_rf_" + str(future) + ".pkl")


def rf_predict(future, set_name, pred_dates_test, X_test, y_test, y_scaler):

    folder_path = os.getcwd()
    model_directory = ""
    csv_directory = ""

    model = load(model_directory + "" + set_name + "_rf_" + str(future) + ".pkl")
    predictions = model.predict(X_test).reshape(-1, 1)
    predictions = y_scaler.inverse_transform(predictions)
    y_test = y_scaler.inverse_transform(y_test)

    make_csvs(csv_directory, predictions, y_test, pred_dates_test, set_name, future, "rf")

    print(f"Finished running rf prediction on future window {0}", future)

    metric_outputs = get_metrics(predictions, y_test, 0, "rf")
    return metric_outputs


def rf_evaluate(future, set_name, X_train, y_train, epochs, epd):

    time_start = time.time()

    folder_path = os.getcwd()
    model_directory = ""

    absl.logging.set_verbosity(absl.logging.ERROR)

    rf_train_model(future, epochs,
            model_directory, set_name, X_train, y_train, epd)
    
    print("Finished evaluating rf for future {0}".format(future))

    time_end = time.time()

    return time_end - time_start


    


In [35]:
def xgb_train_model(future, epochs, model_directory, set_name, X_train, y_train, epd):
    
    split = 0.9

    length = X_train.shape[0]
    X_train_temp = X_train[:int(length * split), :]
    y_train_temp = y_train[:int(length * split), :]
    X_val = X_train[int(length * split):, :]
    y_val = y_train[int(length * split):, :]

    tss = TimeSeriesSplit(n_splits=5, test_size=epd*90, gap=0)
    estimator = xgb.XGBRegressor(booster='gbtree',    
            early_stopping_rounds=50,
            objective='reg:squarederror',
            verbosity=0)

    search_space = {
        "learning_rate": (0.01, 1.0, "log-uniform"),
        "min_child_weight": (0, 10),
        "max_depth": (1, 50),
        "subsample": (0.01, 1.0, "uniform"),
        "colsample_bytree": (0.01, 1.0, "log-uniform"),
        "reg_lambda": (1e-9, 1.0, "log-uniform"),
        "reg_alpha": (1e-9, 1.0, "log-uniform"),
        "gamma": (1e-9, 0.5, "log-uniform"),
        "n_estimators": (5, 5000),
    }

    model = BayesSearchCV(
        estimator=estimator,
        search_spaces=search_space,
        scoring="neg_root_mean_squared_error",
        cv=tss,
        n_jobs=-1,
        n_iter=epochs,
        verbose=0,
        refit=True,
    )

    model = model.fit(X_train_temp, y_train_temp, eval_set=[(X_val, y_val)], verbose=False)
    dump(model, model_directory + "" + set_name + "_xgb_" + str(future) + ".pkl")


def xgb_predict(future, set_name, pred_dates_test, X_test, y_test, y_scaler):

    folder_path = os.getcwd()
    model_directory = ""
    csv_directory = ""

    model = load(model_directory + "" + set_name + "_xgb_" + str(future) + ".pkl")
    predictions = model.predict(X_test).reshape(-1, 1)
    predictions = y_scaler.inverse_transform(predictions)
    y_test = y_scaler.inverse_transform(y_test)

    make_csvs(csv_directory, predictions, y_test, pred_dates_test, set_name, future, "xgb")

    print("Finished running xgb prediction on future window {0}".format(future))

    metric_outputs = get_metrics(predictions, y_test, 0, "xgb")
    return metric_outputs


def xgb_evaluate(future, set_name, X_train, y_train, epochs, epd):

    time_start = time.time()

    model_directory = ""

    absl.logging.set_verbosity(absl.logging.ERROR)

    xgb_train_model(future, epochs,
            model_directory, set_name, X_train, y_train, epd)
    
    print("Finished evaluating xgb for future {0}".format(future))

    time_end = time.time()
    return time_end - time_start
    


In [36]:
# if __name__=="__main__":

try:
    
    csv_directory = ""
    print(csv_directory)
    
    data = pd.read_excel(csv_directory + r'ausdata.xlsx').set_index("Date")
    holidays = pd.read_excel(csv_directory + r'Holidays2.xls')

    data['Holiday'] = data.index.isin(holidays['Date']).astype(int)

    file_name = csv_directory + "matlab_temp.xlsx"
    data.to_excel(file_name)

except FileNotFoundError:

    print("Ausdata and Holidays2 xl files are not present in \"csvs\" directory.")
    print("Ensure they are before continuing")

In [37]:
# if __name__=="__main__":

csv_directory = ""

# These are the values that need changing per different dataset
file_path = "matlab_temp.xlsx"
set_name = "matlab"
target = "SYSLoad"
trend_type = "Additive"
epd = 48
future = 7

cleaning = 1
training = 1
predicting = 1
eval_tpot = 0

partition = 5000
data_epochs = 10

# Don't know what else to put in cleaning parameters tbh
# put more research into this area
cleaning_parameters = {
  # This is seriously jank. It works for now, but golly...
  'pca_dimensions': [None, math.inf, -math.inf],
  'scalers': ['standard', 'minmax']
}

window = 10
split = 0.8
epochs = 10
batch_size = 32

if cleaning:

  data, outputs = feature_adder(csv_directory, file_path, target, trend_type, future, epd,  set_name)

  # Decide on exactly what size this partition should be
  # Essentially this grid search is shit and has to be optimized later on down the track
  # It'll just get the job done for now
  best_results = data_cleaning_pipeline(data[:partition], outputs[:partition], cleaning_parameters, target, split, data_epochs, batch_size, csv_directory)

else:

  if os.path.exists(csv_directory + "best_data_parameters.csv") \
          and os.path.exists(csv_directory + "" + set_name + "_data_" + str(future) + ".csv") \
          and os.path.exists(csv_directory + "" + set_name + "_outputs_" + str(future) + ".csv"):
      
      best_results = pd.read_csv(csv_directory + "best_data_parameters.csv").to_dict('index')
      best_results = best_results.get(0)
      data, outputs = load_datasets(csv_directory, set_name, future)

  else:
      data, outputs = feature_adder(csv_directory, file_path, target, trend_type, future, epd,  set_name)

      # Decide on exactly what size this partition should be
      # Essentially this grid search is shit and has to be optimized later on down the track
      # It'll just get the job done for now
      best_results = data_cleaning_pipeline(data[:partition], outputs[:partition], cleaning_parameters, target, split, data_epochs, batch_size, csv_directory)

print("finished cleaning")
X_frame, y_data, pred_dates, y_scaler = finalise_data(data, outputs, target, best_results)
length = X_frame.shape[0]

pred_dates_test = pred_dates[int(length*split) + window:]

X_2d = create_dataset_2d(X_frame, window)
X_3d = create_dataset_3d(X_frame, window)

y_test = y_data[int(length*split) + window:]
X_test_2d = X_2d[int(length*split):]
X_test_3d = X_3d[int(length*split):]

y_train = y_data[window:int(length*split) + window]
X_train_2d = X_2d[:int(length * split)]
X_train_3d = X_3d[:int(length * split)]

if training:

  np.save("X_train_3d.npy", X_train_3d)

  bnn_time = bnn_evaluate(future, set_name, X_train_2d, y_train, epochs, batch_size, y_scaler, epd)
  cnn_time = cnn_evaluate(future, set_name, X_train_3d, y_train, epochs, batch_size, y_scaler, epd)
  xgb_time = xgb_evaluate(future, set_name, X_train_2d, y_train, epochs, epd)
  rf_time = rf_evaluate(future, set_name, X_train_2d, y_train.reshape(-1), epochs, epd)
  base_time = simple_evaluate(future, set_name, X_train_2d, y_train, epochs, batch_size)

if predicting:

  bnn_metrics = bnn_predict(future, set_name, pred_dates_test, X_test_2d, y_test, y_scaler)
  cnn_metrics = cnn_predict(future, set_name, pred_dates_test, X_test_3d, y_test, y_scaler)
  xgb_metrics = xgb_predict(future, set_name, pred_dates_test, X_test_2d, y_test, y_scaler)
  rf_metrics = rf_predict(future, set_name, pred_dates_test, X_test_2d, y_test, y_scaler)
  base_metrics = simple_predict(future, set_name, pred_dates_test, X_test_2d, y_test, y_scaler)

  if training:
      bnn_metrics['TIME'] = bnn_time
      cnn_metrics['TIME'] = cnn_time
      xgb_metrics['TIME'] = xgb_time
      rf_metrics['TIME'] = rf_time
      base_metrics['TIME'] = base_time

  metrics = [bnn_metrics, cnn_metrics, xgb_metrics, rf_metrics, base_metrics]
  metrics = normalise_metrics(metrics, training)

  metrics = {"Basic_nn": metrics[0], "Complex_nn": metrics[1], "xgb": metrics[2], 
              "rf": metrics[3], "Baseline": metrics[4]}

  make_metrics_csvs(csv_directory, metrics, set_name, future, training)

# if eval_tpot:
#     # Problem still exists with tpot for whatever reason
#     tpot_evaluate(future, set_name, X_train_2d, y_train.reshape(-1), pred_dates_test, X_test_2d, y_test.reshape(-1), y_scaler)

if os.path.exists("X_train_3d.npy"):
  os.remove("X_train_3d.npy")


Saved future window 7 to csvs
Trained scale:standard dim:None
Trained scale:standard dim:inf
Trained scale:standard dim:-inf
Trained scale:minmax dim:None
Trained scale:minmax dim:inf
Trained scale:minmax dim:-inf
finished cleaning


KeyboardInterrupt: ignored